In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
import seaborn as sns

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
print(os.listdir("../input/home-credit-default-risk"))

In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
print('Training data shape: ',app_train.shape)
app_train.head()

In [ ]:
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
print('Testing data size: ', app_test.shape)
app_test.head()

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
plt.hist(app_train['TARGET'])

In [ ]:
sns.countplot(app_train['TARGET'])

In [ ]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_per = 100*(mis_val/len(df))
    mis_val_tab = pd.concat([mis_val,mis_val_per],axis=1)
    mis_val_tab_ren_col = mis_val_tab.rename(
        columns={0:'Missing Values', 1:'% of Total values'}
    )
    mis_val_tab_ren_col = mis_val_tab_ren_col[
        mis_val_tab_ren_col.iloc[:,1]!=0
    ].sort_values('% of Total values', ascending=False).round(1)
    print('Your selected dataframe has '+str(df.shape[1]) + 'columns.\n'
         'There are '+str(mis_val_tab_ren_col.shape[0])+' columns that have missing values.')
    return mis_val_tab_ren_col

In [ ]:
missing_vals = missing_values_table(app_train)
missing_vals.head(20)

In [ ]:
app_train.dtypes.value_counts()

In [ ]:
app_train.select_dtypes('object').apply(pd.Series.nunique,axis=0)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_cnt = 0
for col in app_train:
    if app_train[col].dtype == 'object':
        if len(list(app_train[col].unique())) <=2:
            le.fit(app_train[col])
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            le_cnt+=1
print(f'{le_cnt} columsn were label encoded.')

In [ ]:
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

In [ ]:
train_labels = app_train['TARGET']

app_train, app_test = app_train.align(app_test, join='inner',axis=1)

app_train['TARGET'] = train_labels

print('Training Feature shape: ',app_train.shape)
print('Testing Feature shape: ', app_test.shape)

In [ ]:
(app_train['DAYS_BIRTH']/-365).describe()

In [ ]:
app_train['DAYS_EMPLOYED'].describe()

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram')
plt.xlabel('Days Employment')

In [ ]:
sns.histplot(data = app_train, x='DAYS_EMPLOYED',bins=50000)

In [ ]:
plt.hist(app_train['DAYS_EMPLOYED'])

In [ ]:
anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]
print('The non-anomalies default on %0.2f%% of loans' % 
      (100 * non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

In [ ]:
app_train['DAYS_EMPOLOYED_ANOM'] = app_train['DAYS_EMPLOYED']==365243
app_train['DAYS_EMPLOYED'].replace({365243:np.nan}, inplace=True)
plt.hist(app_train['DAYS_EMPLOYED'])

In [ ]:
app_test['DAYS_EMPLOYED_ANOM'] = app_test['DAYS_EMPLOYED'] == 365243
app_test['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)
print('There are %d anomalies in the test data out of %d entries'
     % (app_test['DAYS_EMPLOYED_ANOM'].sum(),len(app_test)))

In [ ]:
correlations = app_train.corr()['TARGET'].sort_values()

print('Most Posititve Correlation:\n', correlations.tail(15))
print('\nMost Negative Correlation:\n', correlations.head(15))

In [ ]:
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

In [ ]:
plt.style.use('fivethirtyeight')

plt.hist(app_train['DAYS_BIRTH']/365, edgecolor='k', bins=25)
plt.title('Age of Client')
plt.xlabel('Age (years)')
plt.ylabel('Count')

In [ ]:
plt.figure(figsize = (10,8))

sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH']/365, label='target==0')
sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH']/365, label='target==1')
plt.xlabel('Age (years)')
plt.ylabel('Density')
plt.title('Distribution of Ages')

In [ ]:
age_data = app_train[['TARGET','DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH']/365

age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], 
                                  bins=np.linspace(20,70,num=11))
age_data.head(10)

In [ ]:
age_groups = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize=(8,8))

plt.bar(age_groups.index.astype(str), 100*age_groups['TARGET'])

plt.xticks(rotation = 75)
plt.xlabel('Age Group (years)')
plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group')

In [ ]:
ext_data = app_train[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(ext_data_corrs, cmap=plt.cm.RdYlBu_r,vmin=-0.25,annot=True,vmax=0.6)
plt.title('Correlation Heatmap')

In [ ]:
plt.figure(figsize=(10,12))

for i, source in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']):
    plt.subplot(3,1,i+1)
    sns.kdeplot(app_train.loc[app_train['TARGET']==0, source],label='target==0')
    sns.kdeplot(app_train.loc[app_train['TARGET']==1, source],label='target==1')
    plt.title('Distribution of %s by the Target Vaule' % source)
    plt.xlabel('%s' % source)
    plt.ylabel('Density')
plt.tight_layout(h_pad=2.5)

In [ ]:
# Copy the data for plotting
plot_data = ext_data.drop(columns = ['DAYS_BIRTH']).copy()

# Add in the age of the client in years
plot_data['YEARS_BIRTH'] = age_data['YEARS_BIRTH']

# Drop na values and limit to first 100000 rows
plot_data = plot_data.dropna().loc[:100000, :]

# Function to calculate correlation coefficient between two columns
def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.2, .8), xycoords=ax.transAxes,
                size = 20)

# Create the pairgrid object
grid = sns.PairGrid(data = plot_data, diag_sharey=False,
                    hue = 'TARGET', 
                    vars = [x for x in list(plot_data.columns) if x != 'TARGET'])

# Upper is a scatter plot
grid.map_upper(plt.scatter, alpha = 0.2)

# Diagonal is a histogram
grid.map_diag(sns.kdeplot)

# Bottom is density plot
grid.map_lower(sns.kdeplot, cmap = plt.cm.OrRd_r);

plt.suptitle('Ext Source and Age Features Pairs Plot', size = 32, y = 1.05);

In [ ]:
poly_features = app_train[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH','TARGET']]
poly_features_test = app_test[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
poly_target = poly_features['TARGET']
poly_features = poly_features.drop(columns=['TARGET'])
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)

from sklearn.preprocessing import PolynomialFeatures

poly_transform = PolynomialFeatures(degree = 3)

In [ ]:
poly_transform.fit(poly_features)
poly_features = poly_transform.transform(poly_features)
poly_features_test = poly_transform.transform(poly_features_test)
print('Polynomial Features shape: ',poly_features.shape)

In [ ]:
poly_transform.get_feature_names_out(input_features=['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH'])[:25]

In [ ]:
poly_features = pd.DataFrame(poly_features, columns=poly_transform.get_feature_names_out(input_features=['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']))
poly_features['TARGET'] = poly_target
poly_corr = poly_features.corr()['TARGET'].sort_values()
print(poly_corr.head(10))
print(poly_corr.tail(5))

In [ ]:
poly_features_test = pd.DataFrame(poly_features_test, columns=poly_transform.get_feature_names_out(input_features=['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']))
poly_features['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_features, on='SK_ID_CURR', how='left')
poly_features_test['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_features_test, on='SK_ID_CURR', how='left')
app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join='inner',axis=1)
print('Training data with polynomial features shape: ',app_train_poly.shape)
print('Testing data with polynomial features shape: ',app_test_poly.shape)

In [ ]:
app_train_domain = app_train.copy()
app_test_domain = app_test.copy()

app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']

In [ ]:
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']

In [ ]:
plt.figure(figsize = (12, 20))
# iterate through the new features
for i, feature in enumerate(['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']):
    
    # create a new subplot for each source
    plt.subplot(4, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 0, feature], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 1, feature], label = 'target == 1')
    
    # Label the plots
    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
else:
    train = app_train.copy()
features = list(train.columns)
test = app_test.copy()
imputer = SimpleImputer(strategy='median')
scaler = MinMaxScaler(feature_range=(0,1))
imputer.fit(train)
train = imputer.fit_transform(train)
test = imputer.fit_transform(test)
train = scaler.fit_transform(train)
test = scaler.fit_transform(test)
print('Training data shape: ', train.shape)
print('Testing data shape: ',test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C=0.0001)
log_reg.fit(train, train_labels)

In [ ]:
log_reg_pred = log_reg.predict_proba(test)[:, 1]

In [ ]:
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()

In [ ]:
submit.to_csv('log_reg_baseline.csv',index=False)